<a href="https://colab.research.google.com/github/ariakhademi/telematics/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install opendatasets

In [4]:
import opendatasets as od
import pandas as pd
import torch.nn as nn

In [8]:
dataset_url = 'https://www.kaggle.com/datasets/yunlevin/levin-vehicle-telematics'
od.download(dataset_url)
file_path = 'levin-vehicle-telematics/allcars.csv'

# Load the dataset into a pandas DataFrame
df = pd.read_csv(file_path)

Skipping, found downloaded files in "./levin-vehicle-telematics" (use force=True to force download)


<ipython-input-8-07865fba3548>:9: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


**Detect hard brakes with a threshold**

In [15]:
# constants
acceleration_threshold = 0.5

# detect hard brakes
df['timeStamp'] = pd.to_datetime(df['timeStamp'])
delta_speed = df['speed'].diff()
delta_time = df['timeStamp'].diff().dt.total_seconds()
acceleration = delta_speed / delta_time
df['acceleration'] = acceleration
df['hard_brakes'] = (df['acceleration'] < -acceleration_threshold).astype(int)

0    0
1    0
2    0
3    0
4    0
5    0
6    1
7    0
8    1
9    1
Name: hard_brakes, dtype: int64


**Detect hard brakes with machine learning**

In [6]:
# Train an autoencoder to reconstruct the input.
# Anamolies are those that can't be reconstructed
# with low error
class AutoEncoder(nn.Module):
    def __init__(self):
        super(self).__init__()

        # encoder
        encoder = nn.Sequential(
            nn.Linear(),
            nn.ReLU()
        )

        # decoder
        decoder = nn.Sequential(
            nn.ReLU(),
            nn.Linear(),
            nn.ReLU()
        )

    def forward(self, x):
        return self.decoder(self.encoder(x))